In [15]:
from math import exp
import numpy as np

# Problems 6-10

In [16]:
def E(u, v):
    return exp(u) + exp(2*v) + exp(u*v) + u**2 - 2*u*v + 2*v*v - 3*u - 2*v

In [17]:
def del_E(u, v):
    dE_du = exp(u) + v*exp(u*v) + 2*u - 2*v - 3
    dE_dv = 2*exp(2*v) + u*exp(u*v) - 2*u + 4*v - 2
    return (dE_du, dE_dv)

In [18]:
u, v = 0, 0
for r in range(5):
    du, dv = del_E(u, v)
    u -= 0.01*du
    v -= 0.01*dv

In [19]:
E(u, v)

2.8250003566832635

In [21]:
def H_E(u, v):
    return np.array([
        [exp(u)+v*v*exp(u*v)+2, exp(u*v)+u*v*exp(u*v)-2],
        [exp(u*v)+u*v*exp(u*v)-2, 4*exp(2*v)+u*u*exp(u*v)+4],
    ])

In [27]:
u, v = 0, 0
for _ in range(5):
    du, dv = -np.linalg.pinv(H_E(u, v)) @ np.array(del_E(u, v)).reshape(2, 1)
    u, v = u+du[0], v+dv[0]

In [29]:
E(u, v)

2.360823345643139

# Problems 13-15

In [283]:
def gen_data():
    X = np.random.random((1000, 2))
    y = np.sign(X[:,0]**2 + X[:,1]**2 - 0.6)
    subset_ind = np.arange(1000)
    np.random.shuffle(subset_ind)
    for i in subset_ind[:100]:
        y[i] = -1*y[i]
    return np.c_[np.ones(1000), X], y.reshape((1000, 1))

In [284]:
def lin_reg_Ein(X, y):
    X = np.c_[X[:,0], X[:,1], X[:,2], X[:,1]*X[:,2], X[:,1]*X[:,1], X[:,2]*X[:,2]]
    w = np.linalg.pinv(X) @ y.reshape((1000, 1))
#     print(w)
#     return np.average(np.sign(X @ w) != y)
    return w

In [285]:
s = 0
for rep in range(1000):
    w = lin_reg_Ein(*gen_data())
    X, y = gen_data()
    X = np.c_[X[:,0], X[:,1], X[:,2], X[:,1]*X[:,2], X[:,1]*X[:,1], X[:,2]*X[:,2]]
    s += np.average(np.sign(X @ w).reshape(y.shape) != np.sign(y))
print(s/1000)

0.12898000000000015


# Problems 18-20

In [286]:
import pandas as pd

In [287]:
def load_data(file):
    X = pd.read_csv(file, header=None, sep=' ').drop([0], axis=1).to_numpy()
    X, y = X[:,:-1], X[:,-1]
    return np.c_[np.ones(X.shape[0]), X], y

In [288]:
def theta(s):
    return 1/(1+np.exp(-s))

In [289]:
def del_Ein(w, X, y):
    N, d = X.shape
    w = w.reshape(d)
    res = np.zeros(d)
    for i in range(N):
        res += theta(-y[i]*(w @ X[i]))*(-y[i]*X[i])
    return res/N

In [314]:
T = 2000
eta = 0.001
X, y = load_data('train.dat')
N, d = X.shape
w = np.zeros(X.shape[1])
for t in range(T):
    ind = t % N
#     w -= eta * del_Ein(w, X, y)
    w += eta * theta(-y[ind] * (w @ X[ind])) * (y[ind]*X[ind])

In [315]:
Xtest, ytest = load_data('test.dat')
N = Xtest.shape[0]
np.average(np.sign((Xtest @ w).reshape(N)) != ytest)

0.473